v0: DBPIA, Scienceon Rule계산(Yop + co_authorship + keyword)
v1: title 조건 추가, jaro_winkler_metric 추가
v2: site 조건 변경
v3: Rule계산(joc + yop + title_cossim + abstract_cossim) 추가, TF-IDF 추가, 교신저자 비교
v4: co_authorship 비율추가
v5: site 조건 수정, 함수/변수이름 변경, filter 함수 추가, title_cossim + abstract_cossim 삭제, check_college 추가

In [9]:
from bson.objectid import ObjectId
from pymongo import MongoClient
import numpy as np
import jaro
import re

def check_college(univ0):

    client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')
    univName = client['PUBLIC']['CollegeName']
    
    univ1 = re.sub("산학협력단|병원","",univ0)
    univ2 = re.sub("대학교","대학교 ",univ1)
    try:
        univ = univ2.replace(",", "").split()
        univ = list(set(univ))
        univs = "{'$or': ["
        for u in range(len(univ)):
            if univ[-1] == univ[u]:
                univs += "{'inputName': '" + univ[u] + "'}"
            else:
                univs += "{'inputName': '" + univ[u] + "'}, "
        univs += "]}"

        univ_query = univName.find_one(eval(univs))
        
    except SyntaxError as e:
        print(e)
        print(univ0)
        pass

    if univ_query is None:
        print("Search inst None")
        return(univ0)
    else:
        #print("rawInput:[",univ0,"]","queryOutput:" ,univ_query['originalName'])
        return univ_query['originalName'] #univ0, univ_query

def filter(site, rawdata):
    if site == 'NTIS' :
        inst = check_college(rawdata['ldAgency'])
        inst = rawdata['ldAgency']

        coauthor = rawdata['rsc'].split(";")
        year = int(rawdata['prdStart'][:4])
        keyword = rawdata['koKeyword'].split(",")
        journal = ""
        conference = ""
        title = ""
        mng = rawdata['mng']

    else :
        inst = check_college(rawdata['author_inst'].split(";")[rawdata['author'].split(";").index(name)])
        inst = rawdata['author_inst'].split(";")[rawdata['author'].split(";").index(name)]
        coauthor = rawdata['author'].split(";")[1:-1]
        year = int(rawdata['issue_year'][:4])
        keyword = rawdata['paper_keyword'].replace(" ", "").split(".")
        journal = rawdata['journal']
        conference = rawdata['issue_inst']
        title = rawdata['title']
        mng = rawdata['author'].split(";")[-1]
        #abstract = rawdata['abstract']

    return inst, coauthor, year, keyword, journal, conference, title, mng
    #return inst, coauthor, year, keyword, journal, conference, title, mng, abstract

def Secondary_filter(name, site1, raw_one1, site2, raw_one2):
    inst1, coauthor1, year1, keyword1, journal1, conference1, title1, mng1 = filter(site1, raw_one1)
    inst2, coauthor2, year2, keyword2, journal2, conference2, title2, mng2 = filter(site2, raw_one2)
    print(inst1, inst2)
    
    inst = jaro.jaro_winkler_metric(inst1, inst2)
    weight = 0
    joc = 0
    
    if name in coauthor1:
        coauthor1.remove(name)

    if name in coauthor2:
        coauthor2.remove(name)

    co_author_count = len([i for i in coauthor1 if i in coauthor2])

    if site1 != 'NTIS' and site2 != 'NTIS' :
        if title1 == title2 or mng1 == mng2: # or inst >= 0.8:
            weight = 4
            return weight

        else:
            joc = 1 if journal1 == journal2 and conference1 == conference2 else -1
    else:
        if inst >= 0.8:
            weight = 4
            return weight
            
    yop = -(2*(abs(year1-year2)/10)-1)
            
    if len(coauthor1) > len(coauthor2):
        co_author_ratio = co_author_count/len(coauthor1)
    else:
        co_author_ratio = co_author_count/len(coauthor2)
                
    co_authorship = (1 - np.exp(-co_author_count))/2 + (co_author_ratio/2)
    keyword = 1 - np.exp(-len([i for i in keyword1 if i in keyword2]))

    print(f'joc: {joc} | yop: {yop} | co_authorship: {co_authorship} | keyword: {keyword}')

    weight = joc + yop + co_authorship + keyword

    return weight

In [10]:
client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')

name = '이상원'
site1 = 'Scienceon'
site2 = 'KCI'
input_paper1 = ObjectId('61cbfa95597f83aefb455c3a')
input_paper2 = ObjectId('61cbfb6bf72c82d20d455ca1')

if site1 == 'Scienceon' :
    site_raw1 = client['SCIENCEON']['Rawdata']
elif site1 == 'NTIS' :
    site_raw1 = client['NTIS']['Rawdata']
elif site1 == 'DBPIA' :
    site_raw1 = client['DBPIA']['Rawdata']

if site2 == 'Scienceon' :
    site_raw2 = client['SCIENCEON']['Rawdata']
elif site2 == 'NTIS' :
    site_raw2 = client['NTIS']['Rawdata']
elif site2 == 'DBPIA' :
    site_raw2 = client['DBPIA']['Rawdata']

input_raw1 = site_raw1.find_one({ '_id' : input_paper1 })
input_raw2 = site_raw2.find_one({ '_id' : input_paper2 })

Secondary_filter(name, site1, input_raw1, site2, input_raw2)

Search inst None
Department of Interaction Science, Sungkyunkwan University 성균관대학교
0.0


4